# EMI AND LOAN CALCULATION

## Calculate emi taking average of all major banks across 20 years
### average rates :

     | Bank / Lender         | Typical Floating Rate (p.a.)|
     | --------------------- | ------------------ |
     | SBI                   | ~7.50% – 8.70%+ |
     | HDFC Bank             | ~7.90% – 13.20% |
     | ICICI Bank            | ~8.75% – 9.80% |
     | Kotak Mahindra        | ~7.99% – 13.3% |
     | Union Bank of India   | ~7.45% – 10% |
     | Canara Bank           | ~7.40% – 10.25% |
     | Central Bank of India | ~7.35% – 9.40% |
     | Bajaj Housing Finance | ~7.40% – 10.25% |
     | Indian Bank           | ~7.40% – 8.80% |

we take the average rate as : 
# assumed_home_loan_rate = 8.0


### For FY 2025-26 (assessment year 2026-27), the new tax regime (optional under section 115BAC) has lower and simplified tax rates:

     ₹0 – ₹4 L: Nil

     ₹4 L – ₹8 L: 5%

     ₹8 L – ₹12 L: 10%

     ₹12 L – ₹16 L: 15%

     ₹16 L – ₹20 L: 20%

     ₹20 L – ₹24 L: 25%

     Above ₹24 L: 30%

### Under the old tax regime (still allowed if you choose it when filing ITR):

     Section 24(b): Interest deduction

          Up to ₹2,00,000 per year for self-occupied property

          No limit for rented out properties

     Section 80(C): Principal repayment

          Deduction up to ₹1,50,000 per year


# we are gonna be using both the tax regimes and at last we choose the one which is beneficial (as it is still allowed if you choose it when filing ITR )


# what this code does right now:
     How much loan a buyer can realistically take
     Whether a property is affordable given income constraints
     The true EMI burden and its composition (interest vs principal)  
     The risk profile of the purchase before comparing it with renting



## Data Inputs Used

### Property-Level Inputs (from spreadsheet)

     Property price (listing price)

### User / Investor Inputs

     Gross annual income

     Home loan tenure (20 years)

     Bank interest rates (floating, from multiple lenders)

### Regulatory & Market Assumptions

     RBI loan-to-value (LTV) norms

     Conservative EMI-to-income cap

     Floating-rate home loans


firstly we calculate emi ratio i.e emi as a perrcentage of net monthly income


## Before any loan calculation, the model: 
     Computes tax liability under old and new tax regimes Automatically selects the lower tax regime Derives net annual income

## Maximum Loan Eligibility (Income-Constrained)

     Using the EMI ceiling, interest rate, and tenure:

     The model reverse-calculates the maximum loan allowed by income

## Down Payment Modeling (RBI-Compliant)

     Instead of assuming a fixed down payment, the model uses price-based minimum down payment rules aligned with RBI LTV norms:

     | Property Price | Minimum Down Payment |
     | -------------- | -------------------- |
     | ≤ ₹30 lakh     | 10% |
     | ₹30–75 lakh    | 20% |
     | > ₹75 lakh     | 25% |


## Actual Loan Determination (Final Constraint)

     The final loan amount is determined as:

     Minimum of:

     Loan required after down payment

     Maximum loan allowed by income

## EMI Calculation and Amortization

     For the finalized loan:

          Monthly EMI is calculated

          Full amortization schedule is generated

     EMI is split into:

          Interest component

          Principal component

     Additionally:

          Monthly data is aggregated into year-wise EMI splits

## EMI Stress Ratio

EMI as a percentage of net monthly income

Loan Cap Classification

     PROPERTY_CAPPED: Loan limited by property price

     INCOME_CAPPED: Loan limited by income affordability

In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv("kolkata.csv")
print(df.columns.tolist())

['Name', 'Address', 'Bedrooms', 'Price', 'Rent', 'Area', 'Furnishing']


In [2]:
import numpy as np
import pandas as pd

# ============================================================
# 1. TAX CALCULATION (OLD & NEW REGIME)
# ============================================================

def tax_old_regime(taxable_income, rebate_limit=500000):
    slabs = [(250000, 0.0), (500000, 0.05),
             (1000000, 0.20), (float("inf"), 0.30)]
    tax, prev = 0, 0
    for limit, rate in slabs:
        if taxable_income > prev:
            tax += (min(taxable_income, limit) - prev) * rate
            prev = limit
    return 0 if taxable_income <= rebate_limit else tax


def tax_new_regime(taxable_income, rebate_limit=1275000):
    slabs = [
        (400000, 0.0), (800000, 0.05),
        (1200000, 0.10), (1600000, 0.15),
        (2000000, 0.20), (2400000, 0.25),
        (float("inf"), 0.30)
    ]
    tax, prev = 0, 0
    for limit, rate in slabs:
        if taxable_income > prev:
            tax += (min(taxable_income, limit) - prev) * rate
            prev = limit
    return 0 if taxable_income <= rebate_limit else tax


# ============================================================
# 2. HOME LOAN & EMI HELPERS
# ============================================================

def average_home_loan_rate(bank_rates_fp):
    return float(np.mean(bank_rates_fp))


def calculate_emi(loan_amount, annual_rate, tenure_years):
    r = annual_rate / 12 / 100
    n = tenure_years * 12
    return loan_amount * r * (1 + r)**n / ((1 + r)**n - 1)


def loan_from_emi(emi, annual_rate, tenure_years):
    r = annual_rate / 12 / 100
    n = tenure_years * 12
    return emi * ((1 + r)**n - 1) / (r * (1 + r)**n)


# ============================================================
# 3. DOWN PAYMENT (RBI LTV)
# ============================================================

def min_down_payment_pct(property_price):
    if property_price <= 3_000_000:
        return 0.10
    elif property_price <= 7_500_000:
        return 0.20
    else:
        return 0.25


# ============================================================
# 4. RENTING MODEL (REAL RENT)
# ============================================================

def renting_model(
    initial_monthly_rent,
    down_payment,
    monthly_emi,
    rent_escalation_rate,
    investment_return,
    tenure_years
):
    monthly_return = investment_return / 12
    total_months = tenure_years * 12

    monthly_rent = initial_monthly_rent
    sip_corpus = 0.0

    for month in range(1, total_months + 1):
        monthly_sip = max(monthly_emi - monthly_rent, 0)

        if monthly_sip > 0:
            remaining_months = total_months - month + 1
            sip_corpus += monthly_sip * ((1 + monthly_return) ** remaining_months)

        if month % 12 == 0:
            monthly_rent *= (1 + rent_escalation_rate)

    lump_sum_value = down_payment * ((1 + investment_return) ** tenure_years)
    return lump_sum_value + sip_corpus


# ============================================================
# 5. PROPERTY-LEVEL CALCULATION
# ============================================================

def process_property_row(
    property_price,
    initial_monthly_rent,
    gross_annual_income,
    avg_rate,
    tenure_years,
    emi_ratio,
    tax_regime
):
    # ---------- AFFORDABILITY ----------
    net_income = gross_annual_income - min(
        tax_old_regime(gross_annual_income),
        tax_new_regime(gross_annual_income)
    )

    max_emi = (net_income / 12) * emi_ratio
    max_loan = loan_from_emi(max_emi, avg_rate, tenure_years)

    dp_pct = min_down_payment_pct(property_price)
    down_payment = property_price * dp_pct
    loan_amount = min(property_price - down_payment, max_loan)

    monthly_emi = calculate_emi(loan_amount, avg_rate, tenure_years)

    # ---------- BUY SIDE COSTS ----------
    stamp_duty = property_price * 0.07                # 7% upfront
    maintenance_cost = property_price * 0.01 * tenure_years  # 1% yearly
    sale_cost = 0.02                                   # 2% on final value

    final_property_value = property_price * (1.06 ** tenure_years)
    final_property_value *= (1 - sale_cost)

    final_buying_wealth = final_property_value - stamp_duty - maintenance_cost

    # ---------- RENT SIDE ----------
    final_renting_wealth = renting_model(
        initial_monthly_rent,
        down_payment,
        monthly_emi,
        rent_escalation_rate=0.05,
        investment_return=0.10,
        tenure_years=tenure_years
    )

    decision = "RENT" if final_renting_wealth > final_buying_wealth else "BUY"

    return {
        "property_price": property_price,
        "final_buying_wealth_20y": final_buying_wealth,
        "final_renting_wealth_20y": final_renting_wealth,
        "decision": decision
    }


# ============================================================
# 6. RUN FOR ENTIRE CSV
# ============================================================

def run_for_spreadsheet(
    csv_path,
    gross_annual_income,
    bank_rates_fp,
    tenure_years=20,
    emi_ratio=0.40
):
    df = pd.read_csv(csv_path)
    avg_rate = average_home_loan_rate(bank_rates_fp)

    results = []
    for _, row in df.iterrows():
        results.append(
            process_property_row(
                property_price=row["Price"],
                initial_monthly_rent=row["Rent"],
                gross_annual_income=gross_annual_income,
                avg_rate=avg_rate,
                tenure_years=tenure_years,
                emi_ratio=emi_ratio,
                tax_regime="OLD"
            )
        )

    return pd.DataFrame(results)


# ============================================================
# 7. EXECUTION
# ============================================================

if __name__ == "__main__":
    bank_rates_fp = [7.85, 8.0, 8.1, 7.9, 8.2]

    output = run_for_spreadsheet(
        csv_path="kolkata.csv",
        gross_annual_income=18_00_000,
        bank_rates_fp=bank_rates_fp
    )

In [3]:
print(output["decision"].value_counts())

decision
BUY     2489
RENT     959
Name: count, dtype: int64
